In [28]:
import torch
import rasterio
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.optim import SGD, Adam
from torch.nn import BCELoss
from unet_model import UNet
#from CustomSampler import RandomBatchGeoSampler
from torchgeo.samplers import RandomBatchGeoSampler
from torchgeo.datasets import Sentinel2, stack_samples
from torch.utils.tensorboard import SummaryWriter
from RasterDataset import RasterDataset
from torchmetrics.functional import precision_recall
from sklearn.utils import class_weight 
from torch import nn
import kornia.augmentation as K
from typing import Dict, List
%load_ext tensorboard
%tensorboard --logdir=runs2 --port=6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with 255).
Contents of stderr:
TensorFlow installation not found - running with reduced feature set.
[2022-08-18T21:51:37Z ERROR rustboard_core::disk_logdir] While walking log directory: IO error for operation on runs2: No such file or directory (os error 2)

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

E0818 21:51:37.557521 140639171536704 program.py:298] TensorBoard could not bind to port 6006, it was already in use
ERROR: TensorBoard could not bind to port 6006, it was already in use

In [45]:
class MinMaxScale(nn.Module):
    """Normalize channels to the range [0, 1] using min/max values."""

    def __init__(self, min: List[float], max: List[float]) -> None:
        super().__init__()
        self.min = torch.tensor(min)[:, None, None]
        self.max = torch.tensor(max)[:, None, None]
        self.denominator = self.max - self.min

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        x = inputs
        # Batch
        if x.ndim == 4:
            x = (x - self.min[None, ...]) / self.denominator[None, ...]
        # Sample
        else:
            x = (x - self.min) / self.denominator

        inputs = x.clamp(0, 1)
        return inputs
    
class MinMaxClamp(nn.Module):
    """clamp channels to the range [0, 1] using min/max values."""

    def __init__(self, min: List[float], max: List[float]) -> None:
        super().__init__()
        self.min = torch.tensor(min)[:, None, None]
        self.max = torch.tensor(max)[:, None, None]

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        x = inputs
        inputs = torch.clamp(x, self.min, self.max)
        return inputs

In [56]:
num_epochs = 100
learning_rate = 0.001
batch = 32

test_site = '../chopped_data/herschel'
training_sites = ['../chopped_data/herschel', '../chopped_data/horton', '../chopped_data/kolguev', '../chopped_data/lena']
val_sites = ['../chopped_data/herschel_val', '../chopped_data/horton_val', '../chopped_data/kolguev_val', '../chopped_data/lena_val', '../chopped_data/unvalidated']

training_sites.remove(test_site)
val_sites.remove(test_site + '_val')

min_values = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
max_values = [33.0, 456.0, 12472.0, 12112.0, 11816.0, 12208.0, 3537.0]

means = np.array([12.563786043752913, 117.29941220911088, 447.64719382044086, 595.8620172472806, 535.3712012658714, 1806.29808204889, 920.1094782110865])
stds = np.array([3.5185324647403644, 80.96723830709035, 470.13904303313586, 464.48946732700625, 476.7811733893084, 1034.8480903358773, 387.2689357063178])

transforms = torch.nn.Sequential(
    K.Normalize(means.astype(float), stds.astype(float))
    ,MinMaxClamp(min=list(-np.ones(7)) , max=list(np.ones(7)))
    ,MinMaxScale(list(-np.ones(7)), list(np.ones(7)))
        )

# Training
train= RasterDataset(root_list=training_sites)
sampler = RandomBatchGeoSampler(train, size=batch, length=2500, batch_size = batch)
train_dataloader = DataLoader(train, num_workers = 6, batch_sampler=sampler, collate_fn=stack_samples)

# Validation
val = RasterDataset(root_list=val_sites)
sampler = RandomBatchGeoSampler(val, size=64, length=500, batch_size = batch)
val_dataloader = DataLoader(val, num_workers = 6, batch_sampler=sampler, collate_fn=stack_samples)

model = UNet(n_channels =7, n_classes=1).cuda()
optimizer = Adam(model.parameters(), learning_rate)

In [58]:
import matplotlib.pyplot as plt
torch.cuda.empty_cache()
writer = SummaryWriter()
for e in range(num_epochs):
    train_loss = 0.0
    total_precision = 0
    total_recall = 0 
    count = 0
    for sample in tqdm(train_dataloader):
        data = sample["image"]
        labels = data[:,0,:,:]
        img = data[:,1:,:,:]
        
        img = transforms(img.float())
        #print(img)
        
#         red = img[0,4,:,:]
#         green = img[0,3,:,:]
#         blue = img[0,2,:,:]
#         rgb = torch.stack((red, green, blue), dim=2)
#         plt.imshow(rgb.float())
#         plt.show()

        weights = class_weight.compute_class_weight(class_weight = 'balanced', classes = np.unique(labels.numpy().flatten()), y= labels.numpy().flatten())
        weights = weights * np.array([1, 0.2])
        weight_matrix = np.empty((labels.shape[0],labels.shape[1],labels.shape[2]))
        for i in range(labels.shape[0]):
            for j in range(labels.shape[1]):
                for g in range(labels.shape[2]):
                    weight_matrix[i , j , g] = weights[labels[i,j,g]]
                        
        labels, img = labels.cuda(), img.cuda()
        
        pred = torch.squeeze(model(img.float())).cuda()

        loss = BCELoss(weight = torch.Tensor(weight_matrix).cuda())
        output = loss(torch.reshape(pred,labels.size()), labels.float())        

        optimizer.zero_grad()
        output.backward()
        optimizer.step()
        train_loss += output.item()
    
        # 0.5 treshhold
        precision, recall = precision_recall(pred, labels)
        total_precision += precision
        total_recall += recall
        
    valid_loss = 0.0
    model.eval()
    for val_sample in val_dataloader:
        val_loss = BCELoss()
        val_data = val_sample["image"]
        val_labels = val_data[:,0,:,:]
        val_img = val_data[:,1:,:,:]
        
        val_img = transforms(val_img.float())
        
        val_labels, val_img = val_labels.cuda(), val_img.cuda()
        
        val_pred = model(val_img.float()).cuda()
        val_output = val_loss(torch.squeeze(val_pred), val_labels.float())
        valid_loss = val_output.item() * val_data.size(0)
    
    writer.add_scalar('loss/train', train_loss/len(train_dataloader), e)
    writer.add_scalar('recall/train', total_recall / len(train_dataloader), e)
    writer.add_scalar('precision/train', total_precision / len(train_dataloader), e)
    print(f'Epoch {e+1} \t\t Training Precision: {total_precision / len(train_dataloader)} \t\t Training Recall: {total_recall / len(train_dataloader)}')
    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(train_dataloader)} \t\t Validation Loss: {valid_loss / len(val_dataloader)}')

100%|███████████████████████████████████████████████████████████████████████████████████| 78/78 [00:22<00:00,  3.46it/s]


Epoch 1 		 Training Precision: 0.0 		 Training Recall: 0.0
Epoch 1 		 Training Loss: 0.16716576064339814 		 Validation Loss: 0.0261358638604482


100%|███████████████████████████████████████████████████████████████████████████████████| 78/78 [00:22<00:00,  3.50it/s]


Epoch 2 		 Training Precision: 0.11194490641355515 		 Training Recall: 0.04251333698630333
Epoch 2 		 Training Loss: 0.14418531796680048 		 Validation Loss: 0.529345448811849


100%|███████████████████████████████████████████████████████████████████████████████████| 78/78 [00:22<00:00,  3.47it/s]


...

Epoch 98 		 Training Precision: 0.22365160286426544 		 Training Recall: 0.5731419324874878
Epoch 98 		 Training Loss: 0.04987319054093976 		 Validation Loss: 0.020278888940811157


100%|███████████████████████████████████████████████████████████████████████████████████| 78/78 [00:22<00:00,  3.50it/s]


Epoch 99 		 Training Precision: 0.24630047380924225 		 Training Recall: 0.4752441942691803
Epoch 99 		 Training Loss: 0.02442226007592697 		 Validation Loss: 0.026770553986231484


100%|███████████████████████████████████████████████████████████████████████████████████| 78/78 [00:22<00:00,  3.52it/s]


Epoch 100 		 Training Precision: 0.19921784102916718 		 Training Recall: 0.4369370937347412
Epoch 100 		 Training Loss: 0.040703945055914424 		 Validation Loss: 0.2915247599283854


In [22]:
# save model
model_name = 'herschel_unvalidated'
save_path = '../models/' + model_name
torch.save(model.state_dict(), save_path)